# Business Understanding

In an effort to make educational investments less speculative, the US Department of Education has matched information from the student financial aid system with federal tax returns to create the College Scorecard dataset.

This dataset contains a wide range of college features including academics, admissions, student demographics, cost, financial aid, completion %'s, repayment, and earnings. 

This project looks at the 'Percent completed within 4 years at original institution' [COMP_ORIG_YR4_RT] and see if there is a correlation with a combination of selected college features from the dataset.

# Data Understanding

The dataset is large and contains data ranging back from 1998. The dataset is limited to the last five years to reduce the running time for the code below. For every column, 'PrivacySuppressed' values were converted to NaN values and the median value grouped by the instutition's ID replaced NaN values.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
np.random.seed(9)

In [2]:
# Connect to SQLite file
conn = sqlite3.connect('data/database.sqlite')
cur = conn.cursor()

In [3]:
# Create connection and check table names
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")

table_names = cur.fetchall()
table_names

[('Scorecard',)]

In [4]:
# Examine columns

cur.execute("""SELECT sql FROM sqlite_master WHERE type = 'table' AND name = 'Scorecard';""")
scorecard_columns = cur.fetchone()

In [5]:
# Create a dataframe of currently operating schools that exclude
# grad program only, Predominantly certificate-degree granting,
# and not classified schools between the year 2007 and 2012

df = pd.read_sql("""
SELECT * 
FROM Scorecard 
WHERE year BETWEEN 2007 AND 2012
AND CURROPER = "Currently certified as operating"
AND PREDDEG in ("Predominantly bachelor's-degree granting", 
                "Predominantly associate's-degree granting")
;
""", conn, index_col='Id')

In [6]:
# Confirm SQL query year range

df['Year'].unique()

array([2007, 2008, 2009, 2010, 2011, 2012], dtype=int64)

In [7]:
df.shape

(19509, 1730)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19509 entries, 73000 to 116507
Columns: 1730 entries, UNITID to Year
dtypes: float64(191), int64(193), object(1346)
memory usage: 257.6+ MB


In [9]:
# df.head(5)

In [10]:
# Convert all 'PrivacySuppressed' values to NaN values
df = df.replace('PrivacySuppressed', np.nan, regex=True) # All data frame

### [COMP_ORIG_YR4_RT] - Target Variable

The target variable contains NaN's which must be handled with. Upon inspection, many of the colleges with NaN's were community or religious training colleges. The NaN's make up less than 5% so those rows were dropped.


In [11]:
explore1 = df[df['COMP_ORIG_YR4_RT'].isnull()]
explore1['INSTNM'].value_counts()

Carolina Christian College                                            6
Jackson Community College                                             6
Ner Israel Rabbinical College                                         6
Pontifical College Josephinum                                         6
International Baptist College                                         6
Clear Creek Baptist Bible College                                     6
Southern California Seminary                                          6
Ecclesia College                                                      6
Central Yeshiva Tomchei Tmimim Lubavitz                               6
Sitting Bull College                                                  6
Sierra College                                                        6
Berkeley City College                                                 6
Pacific States University                                             6
Williamson Christian College                                    

In [12]:
# Checking the NaN Count and percent
X_NaN = df['COMP_ORIG_YR4_RT'].isna().sum()
X_len = len(df['COMP_ORIG_YR4_RT'])
X_NaN, X_len, X_NaN / X_len

(1276, 19509, 0.0654057101850428)

In [13]:
# Create new dataframe that drops NaN rows bc the percent is low enough
processed_df = df.dropna(subset=['COMP_ORIG_YR4_RT']).copy()
processed_df.shape

(18233, 1730)

# Feature Variables

Many of the feature variables also contain NaN's values in their columns. In order to narrow the search for a strong correlation, columns will be dropped if they contain only NaN's. For other columns that have data for other years, the median for that college will replace the NaN values. Other columns that have outdated or duplicate data will be dropped as well. Finally, any remaining NaN values will be filled in with a KNN imputer. This process will be done for each feature category: school, admissions, academics, student, cost, aid, completion, repayment, and earning features.

In [14]:
# look at columns w NaN's
processed_df.isna().sum()

UNITID                                  0
OPEID                                   0
opeid6                                  0
INSTNM                                  0
CITY                                    0
STABBR                                  0
ZIP                                     0
AccredAgency                        18233
INSTURL                             18233
NPCURL                              18233
sch_deg                                 0
HCM2                                    0
main                                    0
NUMBRANCH                               0
PREDDEG                                 0
HIGHDEG                                 0
CONTROL                                 0
st_fips                                 0
region                                  0
LOCALE                              18233
locale2                             18233
LATITUDE                            18233
LONGITUDE                           18233
CCBASIC                           

In [15]:
# remove columns with all NaN's
perc = 100.0 # Like N %
min_count =  int(((100-perc)/100)*df.shape[0] + 1)
processed_df = processed_df.dropna( axis=1, thresh=min_count)

In [16]:
# Recheck dataset size after dropping 100% NaN columns
processed_df.shape

(18233, 1626)

In [17]:
# Inspect other columns with NaN's and 
processed_df.isna().sum()

UNITID                                  0
OPEID                                   0
opeid6                                  0
INSTNM                                  0
CITY                                    0
STABBR                                  0
ZIP                                     0
sch_deg                                 0
HCM2                                    0
main                                    0
NUMBRANCH                               0
PREDDEG                                 0
HIGHDEG                                 0
CONTROL                                 0
st_fips                                 0
region                                  0
ADM_RATE                             7277
ADM_RATE_ALL                         6720
SATVR25                             10844
SATVR75                             10844
SATMT25                             10755
SATMT75                             10755
SATWR25                             14112
SATWR75                           

In [20]:
irrelevant_columns = ['OPEID','opeid6', 'sch_deg', 'HCM2', 'UGDS_AIANOld', 'UGDS_HISPOld', 'C150_4_AIANOld', 'C150_4_HISPOld', 'C150_L4_AIANOld', 'C150_L4_HISPOld']

# processed_df = processed_df.drop(irrelevant_columns, axis=1)
processed_df.shape

(18233, 1616)

# School Features

In [199]:
# create list of general college info
school_features = ['UNITID', 'INSTNM', 'CITY', 'STABBR', 'ZIP', 'main', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region', 'Year', 'COMP_ORIG_YR4_RT']
school_df = processed_df[school_features].copy()

In [201]:
school_df.isna().sum()

UNITID              0
INSTNM              0
CITY                0
STABBR              0
ZIP                 0
main                0
NUMBRANCH           0
PREDDEG             0
HIGHDEG             0
CONTROL             0
st_fips             0
region              0
Year                0
COMP_ORIG_YR4_RT    0
dtype: int64

In [310]:
school_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18233 entries, 73000 to 116507
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UNITID            18233 non-null  int64 
 1   INSTNM            18233 non-null  object
 2   CITY              18233 non-null  object
 3   STABBR            18233 non-null  object
 4   ZIP               18233 non-null  object
 5   main              18233 non-null  object
 6   NUMBRANCH         18233 non-null  int64 
 7   PREDDEG           18233 non-null  object
 8   HIGHDEG           18233 non-null  object
 9   CONTROL           18233 non-null  object
 10  st_fips           18233 non-null  object
 11  region            18233 non-null  object
 12  Year              18233 non-null  int64 
 13  COMP_ORIG_YR4_RT  18233 non-null  int32 
dtypes: int32(1), int64(3), object(10)
memory usage: 2.0+ MB


In [315]:
school_cont_cols = ['UNITID', 'COMP_ORIG_YR4_RT']

In [318]:
school_cont_df = school_df[school_cont_cols]

In [311]:
school_cat_cols = ['INSTNM', 'CITY', 'STABBR', 'ZIP', 'main', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region', 'Year']

In [313]:
school_dummies = pd.get_dummies(school_df[school_cat_cols], drop_first=True)
school_dummies.shape

(18233, 9716)

In [322]:
updated_school_df = school_cont_df.join(school_dummies)
updated_school_df.shape

(18233, 9718)

# School Correlation

In [323]:
updated_school_df['COMP_ORIG_YR4_RT'] = updated_school_df['COMP_ORIG_YR4_RT'].astype(int)

In [ ]:
corr_matrix = updated_school_df.corr()
school_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
school_top5

In [ ]:
school_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
school_top5_list

# Admissions Features

In [159]:
target = ['UNITID','COMP_ORIG_YR4_RT']

In [30]:
admission_features = ['ADM_RATE', 'ADM_RATE_ALL', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75', 'SATVRMID', 'SATMTMID', 'SATWRMID', 'ACTCM25', 'ACTCM75', 'ACTEN25', 'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTWR25', 'ACTWR75', 'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'ACTWRMID', 'SAT_AVG', 'SAT_AVG_ALL']

In [31]:
target_admission = target + admission_features
admission_df = processed_df[target_admission].copy()
admission_df.isna().sum()

UNITID                  0
COMP_ORIG_YR4_RT        0
ADM_RATE             7277
ADM_RATE_ALL         6720
SATVR25             10844
SATVR75             10844
SATMT25             10755
SATMT75             10755
SATWR25             14112
SATWR75             14112
SATVRMID            10844
SATMTMID            10755
SATWRMID            14112
ACTCM25             10584
ACTCM75             10584
ACTEN25             11738
ACTEN75             11738
ACTMT25             11744
ACTMT75             11744
ACTWR25             17137
ACTWR75             17138
ACTCMMID            10584
ACTENMID            11738
ACTMTMID            11744
ACTWRMID            17138
SAT_AVG              9970
SAT_AVG_ALL          9645
dtype: int64

In [32]:
for i in admission_features:
    admission_df[i] = admission_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

C:\Users\Jesus Baquiax\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [33]:
admission_df.isna().sum()

UNITID                  0
COMP_ORIG_YR4_RT        0
ADM_RATE             6450
ADM_RATE_ALL         5937
SATVR25              9941
SATVR75              9941
SATMT25              9899
SATMT75              9899
SATWR25             12135
SATWR75             12135
SATVRMID             9941
SATMTMID             9899
SATWRMID            12135
ACTCM25              9772
ACTCM75              9772
ACTEN25             10737
ACTEN75             10737
ACTMT25             10749
ACTMT75             10749
ACTWR25             15206
ACTWR75             15206
ACTCMMID             9772
ACTENMID            10737
ACTMTMID            10749
ACTWRMID            15206
SAT_AVG              9377
SAT_AVG_ALL          8990
dtype: int64

In [74]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")
admissions_KNN = pd.DataFrame(imputer.fit_transform(admission_df), columns = list(admission_df.columns))

# Admissions - Correlation

In [88]:
admissions_KNN['COMP_ORIG_YR4_RT'] = admissions_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [208]:
corr_matrix = admissions_KNN.corr()
corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)

COMP_ORIG_YR4_RT    1.000000
SATMTMID            0.025243
SATMT25             0.024867
SAT_AVG_ALL         0.024282
SATMT75             0.023885
ACTEN25             0.023854
Name: COMP_ORIG_YR4_RT, dtype: float64

In [209]:
admission_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
admission_top5_list

['SATMTMID', 'SATMT25', 'SAT_AVG_ALL', 'SATMT75', 'ACTEN25']

# Academics Features

The academic features contain two types of categories:

1) the percentage of degrees awarded in a field of study

2) Whether the program is offered at the institution

## Academics (Program %)

In [101]:
program_percentage = ['PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12', 'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39', 'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46', 'PCIP47', 'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54']

In [215]:
target_percentage =  target + program_percentage
percentage_df = processed_df[target_percentage].copy()
percentage_df.isna().sum()

UNITID              0
COMP_ORIG_YR4_RT    0
PCIP01              0
PCIP03              0
PCIP04              0
PCIP05              0
PCIP09              0
PCIP10              0
PCIP11              0
PCIP12              0
PCIP13              0
PCIP14              0
PCIP15              0
PCIP16              0
PCIP19              0
PCIP22              0
PCIP23              0
PCIP24              0
PCIP25              0
PCIP26              0
PCIP27              0
PCIP29              0
PCIP30              0
PCIP31              0
PCIP38              0
PCIP39              0
PCIP40              0
PCIP41              0
PCIP42              0
PCIP43              0
PCIP44              0
PCIP45              0
PCIP46              0
PCIP47              0
PCIP48              0
PCIP49              0
PCIP50              0
PCIP51              0
PCIP52              0
PCIP54              0
dtype: int64

# Academics (Program %) Correlation

In [210]:
corr_matrix = percentage_df.corr()
corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)

COMP_ORIG_YR4_RT    1.000000
PCIP45              0.385909
PCIP23              0.384487
PCIP54              0.381086
PCIP16              0.319399
PCIP26              0.316918
Name: COMP_ORIG_YR4_RT, dtype: float64

In [211]:
percentage_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
percentage_top5_list

['PCIP45', 'PCIP23', 'PCIP54', 'PCIP16', 'PCIP26']

# Academics (Program Offered)

In [102]:
program_offered = ['CIP01CERT1', 'CIP01CERT2', 'CIP01ASSOC', 'CIP01CERT4', 'CIP01BACHL', 'CIP03CERT1', 'CIP03CERT2', 'CIP03ASSOC', 'CIP03CERT4', 'CIP03BACHL', 'CIP04CERT1', 'CIP04CERT2', 'CIP04ASSOC', 'CIP04CERT4', 'CIP04BACHL', 'CIP05CERT1', 'CIP05CERT2', 'CIP05ASSOC', 'CIP05CERT4', 'CIP05BACHL', 'CIP09CERT1', 'CIP09CERT2', 'CIP09ASSOC', 'CIP09CERT4', 'CIP09BACHL', 'CIP10CERT1', 'CIP10CERT2', 'CIP10ASSOC', 'CIP10CERT4', 'CIP10BACHL', 'CIP11CERT1', 'CIP11CERT2', 'CIP11ASSOC', 'CIP11CERT4', 'CIP11BACHL', 'CIP12CERT1', 'CIP12CERT2', 'CIP12ASSOC', 'CIP12CERT4', 'CIP12BACHL', 'CIP13CERT1', 'CIP13CERT2', 'CIP13ASSOC', 'CIP13CERT4', 'CIP13BACHL', 'CIP14CERT1', 'CIP14CERT2', 'CIP14ASSOC', 'CIP14CERT4', 'CIP14BACHL', 'CIP15CERT1', 'CIP15CERT2', 'CIP15ASSOC', 'CIP15CERT4', 'CIP15BACHL', 'CIP16CERT1', 'CIP16CERT2', 'CIP16ASSOC', 'CIP16CERT4', 'CIP16BACHL', 'CIP19CERT1', 'CIP19CERT2', 'CIP19ASSOC', 'CIP19CERT4', 'CIP19BACHL', 'CIP22CERT1', 'CIP22CERT2', 'CIP22ASSOC', 'CIP22CERT4', 'CIP22BACHL', 'CIP23CERT1', 'CIP23CERT2', 'CIP23ASSOC', 'CIP23CERT4', 'CIP23BACHL', 'CIP24CERT1', 'CIP24CERT2', 'CIP24ASSOC', 'CIP24CERT4', 'CIP24BACHL', 'CIP25CERT1', 'CIP25CERT2', 'CIP25ASSOC', 'CIP25CERT4', 'CIP25BACHL', 'CIP26CERT1', 'CIP26CERT2', 'CIP26ASSOC', 'CIP26CERT4', 'CIP26BACHL', 'CIP27CERT1', 'CIP27CERT2', 'CIP27ASSOC', 'CIP27CERT4', 'CIP27BACHL', 'CIP29CERT1', 'CIP29CERT2', 'CIP29ASSOC', 'CIP29CERT4', 'CIP29BACHL', 'CIP30CERT1', 'CIP30CERT2', 'CIP30ASSOC', 'CIP30CERT4', 'CIP30BACHL', 'CIP31CERT1', 'CIP31CERT2', 'CIP31ASSOC', 'CIP31CERT4', 'CIP31BACHL', 'CIP38CERT1', 'CIP38CERT2', 'CIP38ASSOC', 'CIP38CERT4', 'CIP38BACHL', 'CIP39CERT1', 'CIP39CERT2', 'CIP39ASSOC', 'CIP39CERT4', 'CIP39BACHL', 'CIP40CERT1', 'CIP40CERT2', 'CIP40ASSOC', 'CIP40CERT4', 'CIP40BACHL', 'CIP41CERT1', 'CIP41CERT2', 'CIP41ASSOC', 'CIP41CERT4', 'CIP41BACHL', 'CIP42CERT1', 'CIP42CERT2', 'CIP42ASSOC', 'CIP42CERT4', 'CIP42BACHL', 'CIP43CERT1', 'CIP43CERT2', 'CIP43ASSOC', 'CIP43CERT4', 'CIP43BACHL', 'CIP44CERT1', 'CIP44CERT2', 'CIP44ASSOC', 'CIP44CERT4', 'CIP44BACHL', 'CIP45CERT1', 'CIP45CERT2', 'CIP45ASSOC', 'CIP45CERT4', 'CIP45BACHL', 'CIP46CERT1', 'CIP46CERT2', 'CIP46ASSOC', 'CIP46CERT4', 'CIP46BACHL', 'CIP47CERT1', 'CIP47CERT2', 'CIP47ASSOC', 'CIP47CERT4', 'CIP47BACHL', 'CIP48CERT1', 'CIP48CERT2', 'CIP48ASSOC', 'CIP48CERT4', 'CIP48BACHL', 'CIP49CERT1', 'CIP49CERT2', 'CIP49ASSOC', 'CIP49CERT4', 'CIP49BACHL', 'CIP50CERT1', 'CIP50CERT2', 'CIP50ASSOC', 'CIP50CERT4', 'CIP50BACHL', 'CIP51CERT1', 'CIP51CERT2', 'CIP51ASSOC', 'CIP51CERT4', 'CIP51BACHL', 'CIP52CERT1', 'CIP52CERT2', 'CIP52ASSOC', 'CIP52CERT4', 'CIP52BACHL', 'CIP54CERT1', 'CIP54CERT2', 'CIP54ASSOC', 'CIP54CERT4', 'CIP54BACHL']

In [214]:
target_offered = target + program_offered
offered_df = processed_df[target_offered].copy()
offered_df.isna().sum()

UNITID              0
COMP_ORIG_YR4_RT    0
CIP01CERT1          0
CIP01CERT2          0
CIP01ASSOC          0
CIP01CERT4          0
CIP01BACHL          0
CIP03CERT1          0
CIP03CERT2          0
CIP03ASSOC          0
CIP03CERT4          0
CIP03BACHL          0
CIP04CERT1          0
CIP04CERT2          0
CIP04ASSOC          0
CIP04CERT4          0
CIP04BACHL          0
CIP05CERT1          0
CIP05CERT2          0
CIP05ASSOC          0
CIP05CERT4          0
CIP05BACHL          0
CIP09CERT1          0
CIP09CERT2          0
CIP09ASSOC          0
CIP09CERT4          0
CIP09BACHL          0
CIP10CERT1          0
CIP10CERT2          0
CIP10ASSOC          0
CIP10CERT4          0
CIP10BACHL          0
CIP11CERT1          0
CIP11CERT2          0
CIP11ASSOC          0
CIP11CERT4          0
CIP11BACHL          0
CIP12CERT1          0
CIP12CERT2          0
CIP12ASSOC          0
CIP12CERT4          0
CIP12BACHL          0
CIP13CERT1          0
CIP13CERT2          0
CIP13ASSOC          0
CIP13CERT4

In [279]:
offered_df = offered_df.astype(str)

In [281]:
offered_df['CIP01CERT4'].value_counts()

0    17860
1      371
2        2
Name: CIP01CERT4, dtype: int64

In [282]:
dummies = pd.get_dummies(offered_df[program_offered], drop_first=True)

# Academics (Program Offered) Correlation

In [302]:
dummies.shape

(18233, 380)

In [303]:
dummies_df = dummies.join(offered_df[target])

In [304]:
dummies_df.shape

(18233, 382)

In [306]:
dummies_df['COMP_ORIG_YR4_RT'] = dummies_df['COMP_ORIG_YR4_RT'].astype(float)

In [307]:
dummies_df['COMP_ORIG_YR4_RT'] = dummies_df['COMP_ORIG_YR4_RT'].astype(int)

In [308]:
corr_matrix = dummies_df.corr()
offered_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
offered_top5

COMP_ORIG_YR4_RT    1.000000
CIP51BACHL_1        0.023110
CIP26BACHL_1        0.008103
CIP39CERT4_2       -0.000233
CIP25CERT4_2       -0.000233
CIP29CERT4_2       -0.000233
Name: COMP_ORIG_YR4_RT, dtype: float64

In [309]:
offered_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
offered_top5_list

['CIP51BACHL_1',
 'CIP26BACHL_1',
 'CIP39CERT4_2',
 'CIP25CERT4_2',
 'CIP29CERT4_2']

# Student Features

In [216]:
student = ['UGDS', 'UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITENH', 'UGDS_BLACKNH', 'UGDS_API']

In [220]:
target_student = target + student
student_df = processed_df[target_student].copy()
student_df.isna().sum()

UNITID                 0
COMP_ORIG_YR4_RT       0
UGDS                  10
UGDS_WHITE          2859
UGDS_BLACK          2859
UGDS_HISP           2859
UGDS_ASIAN          2859
UGDS_AIAN           2859
UGDS_NHPI           2859
UGDS_2MOR           2859
UGDS_NRA              10
UGDS_UNKN             10
UGDS_WHITENH        9574
UGDS_BLACKNH        9574
UGDS_API            9574
dtype: int64

In [221]:
for i in student:
    student_df[i] = student_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

In [222]:
student_df.isna().sum()

UNITID                0
COMP_ORIG_YR4_RT      0
UGDS                  9
UGDS_WHITE           50
UGDS_BLACK           50
UGDS_HISP            50
UGDS_ASIAN           50
UGDS_AIAN            50
UGDS_NHPI            50
UGDS_2MOR            50
UGDS_NRA              9
UGDS_UNKN             9
UGDS_WHITENH        977
UGDS_BLACKNH        977
UGDS_API            977
dtype: int64

In [223]:
imputer2 = KNNImputer(n_neighbors=2, weights="uniform")
student_KNN = pd.DataFrame(imputer2.fit_transform(student_df), columns = list(student_df.columns))

In [225]:
student_KNN.isna().sum()

UNITID              0
COMP_ORIG_YR4_RT    0
UGDS                0
UGDS_WHITE          0
UGDS_BLACK          0
UGDS_HISP           0
UGDS_ASIAN          0
UGDS_AIAN           0
UGDS_NHPI           0
UGDS_2MOR           0
UGDS_NRA            0
UGDS_UNKN           0
UGDS_WHITENH        0
UGDS_BLACKNH        0
UGDS_API            0
dtype: int64

# Student Correlation

In [226]:
student_KNN['COMP_ORIG_YR4_RT'] = student_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [227]:
corr_matrix = student_KNN.corr()
student_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
student_top5

COMP_ORIG_YR4_RT    1.000000
UGDS_NRA            0.063676
UGDS_ASIAN          0.027090
UGDS_UNKN           0.016609
UGDS_API            0.010668
UGDS                0.004897
Name: COMP_ORIG_YR4_RT, dtype: float64

In [228]:
student_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
student_top5_list

['UGDS_NRA', 'UGDS_ASIAN', 'UGDS_UNKN', 'UGDS_API', 'UGDS']

# Cost Features

In [229]:
cost_list = ['NPT4_PUB', 'NPT4_PRIV', 'NPT4_PROG', 'NPT4_OTHER', 'NPT41_PUB', 'NPT42_PUB', 'NPT43_PUB', 'NPT44_PUB', 'NPT45_PUB', 'NPT41_PRIV', 'NPT42_PRIV', 'NPT43_PRIV', 'NPT44_PRIV', 'NPT45_PRIV', 'NPT41_PROG', 'NPT42_PROG', 'NPT43_PROG', 'NPT44_PROG', 'NPT45_PROG', 'NPT41_OTHER', 'NPT42_OTHER', 'NPT43_OTHER', 'NPT44_OTHER', 'NPT45_OTHER', 'NPT4_048_PUB', 'NPT4_048_PRIV', 'NPT4_048_PROG', 'NPT4_048_OTHER', 'NPT4_3075_PUB', 'NPT4_3075_PRIV', 'NPT4_75UP_PUB', 'NPT4_75UP_PRIV', 'NPT4_3075_PROG', 'NPT4_3075_OTHER', 'NPT4_75UP_PROG', 'NPT4_75UP_OTHER', 'NUM4_PUB', 'NUM4_PRIV', 'NUM4_PROG', 'NUM4_OTHER', 'NUM41_PUB', 'NUM42_PUB', 'NUM43_PUB', 'NUM44_PUB', 'NUM45_PUB', 'NUM41_PRIV', 'NUM42_PRIV', 'NUM43_PRIV', 'NUM44_PRIV', 'NUM45_PRIV', 'NUM41_PROG', 'NUM42_PROG', 'NUM43_PROG', 'NUM44_PROG', 'NUM45_PROG', 'NUM41_OTHER', 'NUM42_OTHER', 'NUM43_OTHER', 'NUM44_OTHER', 'NUM45_OTHER', 'COSTT4_A', 'COSTT4_P', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'TUITIONFEE_PROG']
target_cost = target + cost_list
cost_df = processed_df[target_cost].copy()
cost_df.isna().sum()

UNITID                  0
COMP_ORIG_YR4_RT        0
NPT4_PUB            13317
NPT4_PRIV           11284
NPT4_PROG           18149
NPT4_OTHER          18040
NPT41_PUB           13318
NPT42_PUB           13584
NPT43_PUB           13673
NPT44_PUB           13932
NPT45_PUB           14300
NPT41_PRIV          11356
NPT42_PRIV          11636
NPT43_PRIV          11862
NPT44_PRIV          12424
NPT45_PRIV          13025
NPT41_PROG          18149
NPT42_PROG          18149
NPT43_PROG          18149
NPT44_PROG          18150
NPT45_PROG          18150
NPT41_OTHER         18041
NPT42_OTHER         18059
NPT43_OTHER         18075
NPT44_OTHER         18101
NPT45_OTHER         18119
NPT4_048_PUB        13317
NPT4_048_PRIV       11314
NPT4_048_PROG       18149
NPT4_048_OTHER      18040
NPT4_3075_PUB       13572
NPT4_3075_PRIV      11531
NPT4_75UP_PUB       13911
NPT4_75UP_PRIV      12306
NPT4_3075_PROG      18151
NPT4_3075_OTHER     18054
NPT4_75UP_PROG      18178
NPT4_75UP_OTHER     18096
NUM4_PUB    

In [230]:
for i in cost_list:
    cost_df[i] = cost_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

C:\Users\Jesus Baquiax\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [231]:
cost_df.isna().sum()

UNITID                  0
COMP_ORIG_YR4_RT        0
NPT4_PUB            10915
NPT4_PRIV            7865
NPT4_PROG           17977
NPT4_OTHER          17722
NPT41_PUB           10921
NPT42_PUB           11174
NPT43_PUB           11297
NPT44_PUB           11450
NPT45_PUB           11792
NPT41_PRIV           7903
NPT42_PRIV           8058
NPT43_PRIV           8272
NPT44_PRIV           8770
NPT45_PRIV           9461
NPT41_PROG          17977
NPT42_PROG          17977
NPT43_PROG          17977
NPT44_PROG          17980
NPT45_PROG          17980
NPT41_OTHER         17722
NPT42_OTHER         17744
NPT43_OTHER         17756
NPT44_OTHER         17807
NPT45_OTHER         17846
NPT4_048_PUB        10915
NPT4_048_PRIV        7879
NPT4_048_PROG       17977
NPT4_048_OTHER      17722
NPT4_3075_PUB       11159
NPT4_3075_PRIV       8012
NPT4_75UP_PUB       11434
NPT4_75UP_PRIV       8652
NPT4_3075_PROG      17983
NPT4_3075_OTHER     17734
NPT4_75UP_PROG      18056
NPT4_75UP_OTHER     17794
NUM4_PUB    

In [233]:
imputer3 = KNNImputer(n_neighbors=2, weights="uniform")
cost_KNN = pd.DataFrame(imputer3.fit_transform(cost_df), columns = list(cost_df.columns))

# Cost Correlation

In [237]:
corr_matrix = cost_KNN.corr()
cost_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
cost_top5

COMP_ORIG_YR4_RT    1.000000
TUITIONFEE_OUT      0.670658
COSTT4_A            0.626523
TUITIONFEE_IN       0.621967
NPT45_PUB           0.579966
NPT4_75UP_PUB       0.573320
Name: COMP_ORIG_YR4_RT, dtype: float64

In [241]:
cost_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
cost_top5_list

['TUITIONFEE_OUT', 'COSTT4_A', 'TUITIONFEE_IN', 'NPT45_PUB', 'NPT4_75UP_PUB']

# Aid Features

In [160]:
aid_features = ['PCTPELL', 'PCTFLOAN', 'DEBT_MDN', 'GRAD_DEBT_MDN', 'WDRAW_DEBT_MDN', 'LO_INC_DEBT_MDN', 'MD_INC_DEBT_MDN', 'HI_INC_DEBT_MDN', 'DEP_DEBT_MDN', 'IND_DEBT_MDN', 'PELL_DEBT_MDN', 'NOPELL_DEBT_MDN', 'FEMALE_DEBT_MDN', 'MALE_DEBT_MDN', 'FIRSTGEN_DEBT_MDN', 'NOTFIRSTGEN_DEBT_MDN', 'DEBT_N', 'GRAD_DEBT_N', 'WDRAW_DEBT_N', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N', 'DEP_DEBT_N', 'IND_DEBT_N', 'PELL_DEBT_N', 'NOPELL_DEBT_N', 'FEMALE_DEBT_N', 'MALE_DEBT_N', 'FIRSTGEN_DEBT_N', 'NOTFIRSTGEN_DEBT_N', 'GRAD_DEBT_MDN10YR', 'CUML_DEBT_N', 'CUML_DEBT_P90', 'CUML_DEBT_P75', 'CUML_DEBT_P25', 'CUML_DEBT_P10', 'DEBT_MDN_SUPP', 'GRAD_DEBT_MDN_SUPP', 'GRAD_DEBT_MDN10YR_SUPP']
target_aid = target + aid_features
aid_df = processed_df[target_aid].copy()
aid_df.isna().sum()

UNITID                       0
COMP_ORIG_YR4_RT             0
PCTPELL                   2866
PCTFLOAN                  5732
DEBT_MDN                   605
GRAD_DEBT_MDN             1165
WDRAW_DEBT_MDN             663
LO_INC_DEBT_MDN           2748
MD_INC_DEBT_MDN           1425
HI_INC_DEBT_MDN           4882
DEP_DEBT_MDN              3388
IND_DEBT_MDN              5268
PELL_DEBT_MDN             2723
NOPELL_DEBT_MDN           4412
FEMALE_DEBT_MDN           1867
MALE_DEBT_MDN             3739
FIRSTGEN_DEBT_MDN         1125
NOTFIRSTGEN_DEBT_MDN       750
DEBT_N                     542
GRAD_DEBT_N                783
WDRAW_DEBT_N               590
LO_INC_DEBT_N             2576
MD_INC_DEBT_N              995
HI_INC_DEBT_N             4380
DEP_DEBT_N                2839
IND_DEBT_N                4812
PELL_DEBT_N               2519
NOPELL_DEBT_N             4189
FEMALE_DEBT_N             1660
MALE_DEBT_N               3463
FIRSTGEN_DEBT_N            979
NOTFIRSTGEN_DEBT_N         569
GRAD_DEB

In [161]:
for i in aid_features:
    aid_df[i] = aid_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

C:\Users\Jesus Baquiax\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [155]:
aid_df.isna().sum()

UNITID                       0
INSTNM                       0
CITY                         0
STABBR                       0
ZIP                          0
main                         0
NUMBRANCH                    0
PREDDEG                      0
HIGHDEG                      0
CONTROL                      0
st_fips                      0
region                       0
Year                         0
COMP_ORIG_YR4_RT             0
PCTPELL                     57
PCTFLOAN                   123
DEBT_MDN                   498
GRAD_DEBT_MDN              700
WDRAW_DEBT_MDN             596
LO_INC_DEBT_MDN           1342
MD_INC_DEBT_MDN            886
HI_INC_DEBT_MDN           2661
DEP_DEBT_MDN              1934
IND_DEBT_MDN              2923
PELL_DEBT_MDN             1181
NOPELL_DEBT_MDN           2102
FEMALE_DEBT_MDN           1059
MALE_DEBT_MDN             1987
FIRSTGEN_DEBT_MDN          839
NOTFIRSTGEN_DEBT_MDN       646
DEBT_N                     498
GRAD_DEBT_N                425
WDRAW_DE

In [162]:
imputer4 = KNNImputer(n_neighbors=2, weights="uniform")
AID_KNN = pd.DataFrame(imputer4.fit_transform(aid_df), columns = list(aid_df.columns))

# Aid Correlation

In [163]:
AID_KNN['COMP_ORIG_YR4_RT'] = AID_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [243]:
corr_matrix = AID_KNN.corr()
AID_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
AID_top5

COMP_ORIG_YR4_RT    1.000000
CUML_DEBT_P10       0.209088
CUML_DEBT_P25       0.095919
DEBT_MDN            0.026732
IND_DEBT_MDN        0.006619
UNITID              0.003471
Name: COMP_ORIG_YR4_RT, dtype: float64

In [244]:
aid_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
aid_top5_list

['CUML_DEBT_P10', 'CUML_DEBT_P25', 'DEBT_MDN', 'IND_DEBT_MDN', 'UNITID']

# Completion Features

In [273]:
completion_features = ['C150_4', 'C150_L4', 'D150_4', 'D150_L4', 'C150_4_WHITE', 'C150_4_BLACK', 'C150_4_HISP', 'C150_4_ASIAN', 'C150_4_AIAN', 'C150_4_NHPI', 'C150_4_2MOR', 'C150_4_NRA', 'C150_4_UNKN', 'C150_4_WHITENH', 'C150_4_BLACKNH', 'C150_4_API', 'C150_L4_WHITE', 'C150_L4_BLACK', 'C150_L4_HISP', 'C150_L4_ASIAN', 'C150_L4_AIAN', 'C150_L4_NHPI', 'C150_L4_2MOR', 'C150_L4_NRA', 'C150_L4_UNKN', 'C150_L4_WHITENH', 'C150_L4_BLACKNH', 'C150_L4_API', 'DEATH_YR2_RT', 'COMP_ORIG_YR2_RT', 'COMP_4YR_TRANS_YR2_RT', 'COMP_2YR_TRANS_YR2_RT', 'WDRAW_ORIG_YR2_RT', 'WDRAW_4YR_TRANS_YR2_RT', 'WDRAW_2YR_TRANS_YR2_RT', 'ENRL_ORIG_YR2_RT', 'ENRL_4YR_TRANS_YR2_RT', 'ENRL_2YR_TRANS_YR2_RT', 'UNKN_ORIG_YR2_RT', 'UNKN_4YR_TRANS_YR2_RT', 'UNKN_2YR_TRANS_YR2_RT', 'LO_INC_DEATH_YR2_RT', 'LO_INC_COMP_ORIG_YR2_RT', 'LO_INC_COMP_4YR_TRANS_YR2_RT', 'LO_INC_COMP_2YR_TRANS_YR2_RT', 'LO_INC_WDRAW_ORIG_YR2_RT', 'LO_INC_WDRAW_4YR_TRANS_YR2_RT', 'LO_INC_WDRAW_2YR_TRANS_YR2_RT', 'LO_INC_ENRL_ORIG_YR2_RT', 'LO_INC_ENRL_4YR_TRANS_YR2_RT', 'LO_INC_ENRL_2YR_TRANS_YR2_RT', 'LO_INC_UNKN_ORIG_YR2_RT', 'LO_INC_UNKN_4YR_TRANS_YR2_RT', 'LO_INC_UNKN_2YR_TRANS_YR2_RT', 'MD_INC_DEATH_YR2_RT', 'MD_INC_COMP_ORIG_YR2_RT', 'MD_INC_COMP_4YR_TRANS_YR2_RT', 'MD_INC_COMP_2YR_TRANS_YR2_RT', 'MD_INC_WDRAW_ORIG_YR2_RT', 'MD_INC_WDRAW_4YR_TRANS_YR2_RT', 'MD_INC_WDRAW_2YR_TRANS_YR2_RT', 'MD_INC_ENRL_ORIG_YR2_RT', 'MD_INC_ENRL_4YR_TRANS_YR2_RT', 'MD_INC_ENRL_2YR_TRANS_YR2_RT', 'MD_INC_UNKN_ORIG_YR2_RT', 'MD_INC_UNKN_4YR_TRANS_YR2_RT', 'MD_INC_UNKN_2YR_TRANS_YR2_RT', 'HI_INC_DEATH_YR2_RT', 'HI_INC_COMP_ORIG_YR2_RT', 'HI_INC_COMP_4YR_TRANS_YR2_RT', 'HI_INC_COMP_2YR_TRANS_YR2_RT', 'HI_INC_WDRAW_ORIG_YR2_RT', 'HI_INC_WDRAW_4YR_TRANS_YR2_RT', 'HI_INC_WDRAW_2YR_TRANS_YR2_RT', 'HI_INC_ENRL_ORIG_YR2_RT', 'HI_INC_ENRL_4YR_TRANS_YR2_RT', 'HI_INC_ENRL_2YR_TRANS_YR2_RT', 'HI_INC_UNKN_ORIG_YR2_RT', 'HI_INC_UNKN_4YR_TRANS_YR2_RT', 'HI_INC_UNKN_2YR_TRANS_YR2_RT', 'DEP_DEATH_YR2_RT', 'DEP_COMP_ORIG_YR2_RT', 'DEP_COMP_4YR_TRANS_YR2_RT', 'DEP_COMP_2YR_TRANS_YR2_RT', 'DEP_WDRAW_ORIG_YR2_RT', 'DEP_WDRAW_4YR_TRANS_YR2_RT', 'DEP_WDRAW_2YR_TRANS_YR2_RT', 'DEP_ENRL_ORIG_YR2_RT', 'DEP_ENRL_4YR_TRANS_YR2_RT', 'DEP_ENRL_2YR_TRANS_YR2_RT', 'DEP_UNKN_ORIG_YR2_RT', 'DEP_UNKN_4YR_TRANS_YR2_RT', 'DEP_UNKN_2YR_TRANS_YR2_RT', 'IND_DEATH_YR2_RT', 'IND_COMP_ORIG_YR2_RT', 'IND_COMP_4YR_TRANS_YR2_RT', 'IND_COMP_2YR_TRANS_YR2_RT', 'IND_WDRAW_ORIG_YR2_RT', 'IND_WDRAW_4YR_TRANS_YR2_RT', 'IND_WDRAW_2YR_TRANS_YR2_RT', 'IND_ENRL_ORIG_YR2_RT', 'IND_ENRL_4YR_TRANS_YR2_RT', 'IND_ENRL_2YR_TRANS_YR2_RT', 'IND_UNKN_ORIG_YR2_RT', 'IND_UNKN_4YR_TRANS_YR2_RT', 'IND_UNKN_2YR_TRANS_YR2_RT', 'FEMALE_DEATH_YR2_RT', 'FEMALE_COMP_ORIG_YR2_RT', 'FEMALE_COMP_4YR_TRANS_YR2_RT', 'FEMALE_COMP_2YR_TRANS_YR2_RT', 'FEMALE_WDRAW_ORIG_YR2_RT', 'FEMALE_WDRAW_4YR_TRANS_YR2_RT', 'FEMALE_WDRAW_2YR_TRANS_YR2_RT', 'FEMALE_ENRL_ORIG_YR2_RT', 'FEMALE_ENRL_4YR_TRANS_YR2_RT', 'FEMALE_ENRL_2YR_TRANS_YR2_RT', 'FEMALE_UNKN_ORIG_YR2_RT', 'FEMALE_UNKN_4YR_TRANS_YR2_RT', 'FEMALE_UNKN_2YR_TRANS_YR2_RT', 'MALE_DEATH_YR2_RT', 'MALE_COMP_ORIG_YR2_RT', 'MALE_COMP_4YR_TRANS_YR2_RT', 'MALE_COMP_2YR_TRANS_YR2_RT', 'MALE_WDRAW_ORIG_YR2_RT', 'MALE_WDRAW_4YR_TRANS_YR2_RT', 'MALE_WDRAW_2YR_TRANS_YR2_RT', 'MALE_ENRL_ORIG_YR2_RT', 'MALE_ENRL_4YR_TRANS_YR2_RT', 'MALE_ENRL_2YR_TRANS_YR2_RT', 'MALE_UNKN_ORIG_YR2_RT', 'MALE_UNKN_4YR_TRANS_YR2_RT', 'MALE_UNKN_2YR_TRANS_YR2_RT', 'PELL_DEATH_YR2_RT', 'PELL_COMP_ORIG_YR2_RT', 'PELL_COMP_4YR_TRANS_YR2_RT', 'PELL_COMP_2YR_TRANS_YR2_RT', 'PELL_WDRAW_ORIG_YR2_RT', 'PELL_WDRAW_4YR_TRANS_YR2_RT', 'PELL_WDRAW_2YR_TRANS_YR2_RT', 'PELL_ENRL_ORIG_YR2_RT', 'PELL_ENRL_4YR_TRANS_YR2_RT', 'PELL_ENRL_2YR_TRANS_YR2_RT', 'PELL_UNKN_ORIG_YR2_RT', 'PELL_UNKN_4YR_TRANS_YR2_RT', 'PELL_UNKN_2YR_TRANS_YR2_RT', 'NOPELL_DEATH_YR2_RT', 'NOPELL_COMP_ORIG_YR2_RT', 'NOPELL_COMP_4YR_TRANS_YR2_RT', 'NOPELL_COMP_2YR_TRANS_YR2_RT', 'NOPELL_WDRAW_ORIG_YR2_RT', 'NOPELL_WDRAW_4YR_TRANS_YR2_RT', 'NOPELL_WDRAW_2YR_TRANS_YR2_RT', 'NOPELL_ENRL_ORIG_YR2_RT', 'NOPELL_ENRL_4YR_TRANS_YR2_RT', 'NOPELL_ENRL_2YR_TRANS_YR2_RT', 'NOPELL_UNKN_ORIG_YR2_RT', 'NOPELL_UNKN_4YR_TRANS_YR2_RT', 'NOPELL_UNKN_2YR_TRANS_YR2_RT', 'LOAN_DEATH_YR2_RT', 'LOAN_COMP_ORIG_YR2_RT', 'LOAN_COMP_4YR_TRANS_YR2_RT', 'LOAN_COMP_2YR_TRANS_YR2_RT', 'LOAN_WDRAW_ORIG_YR2_RT', 'LOAN_WDRAW_4YR_TRANS_YR2_RT', 'LOAN_WDRAW_2YR_TRANS_YR2_RT', 'LOAN_ENRL_ORIG_YR2_RT', 'LOAN_ENRL_4YR_TRANS_YR2_RT', 'LOAN_ENRL_2YR_TRANS_YR2_RT', 'LOAN_UNKN_ORIG_YR2_RT', 'LOAN_UNKN_4YR_TRANS_YR2_RT', 'LOAN_UNKN_2YR_TRANS_YR2_RT', 'NOLOAN_DEATH_YR2_RT', 'NOLOAN_COMP_ORIG_YR2_RT', 'NOLOAN_COMP_4YR_TRANS_YR2_RT', 'NOLOAN_COMP_2YR_TRANS_YR2_RT', 'NOLOAN_WDRAW_ORIG_YR2_RT', 'NOLOAN_WDRAW_4YR_TRANS_YR2_RT', 'NOLOAN_WDRAW_2YR_TRANS_YR2_RT', 'NOLOAN_ENRL_ORIG_YR2_RT', 'NOLOAN_ENRL_4YR_TRANS_YR2_RT', 'NOLOAN_ENRL_2YR_TRANS_YR2_RT', 'NOLOAN_UNKN_ORIG_YR2_RT', 'NOLOAN_UNKN_4YR_TRANS_YR2_RT', 'NOLOAN_UNKN_2YR_TRANS_YR2_RT', 'FIRSTGEN_DEATH_YR2_RT', 'FIRSTGEN_COMP_ORIG_YR2_RT', 'FIRSTGEN_COMP_4YR_TRANS_YR2_RT', 'FIRSTGEN_COMP_2YR_TRANS_YR2_RT', 'FIRSTGEN_WDRAW_ORIG_YR2_RT', 'FIRSTGEN_WDRAW_4YR_TRANS_YR2_RT', 'FIRSTGEN_WDRAW_2YR_TRANS_YR2_RT', 'FIRSTGEN_ENRL_ORIG_YR2_RT', 'FIRSTGEN_ENRL_4YR_TRANS_YR2_RT', 'FIRSTGEN_ENRL_2YR_TRANS_YR2_RT', 'FIRSTGEN_UNKN_ORIG_YR2_RT', 'FIRSTGEN_UNKN_4YR_TRANS_YR2_RT', 'FIRSTGEN_UNKN_2YR_TRANS_YR2_RT', 'NOT1STGEN_DEATH_YR2_RT', 'NOT1STGEN_COMP_ORIG_YR2_RT', 'NOT1STGEN_COMP_4YR_TRANS_YR2_RT', 'NOT1STGEN_COMP_2YR_TRANS_YR2_RT', 'NOT1STGEN_WDRAW_ORIG_YR2_RT', 'NOT1STGEN_WDRAW_4YR_TRANS_YR2_RT', 'NOT1STGEN_WDRAW_2YR_TRANS_YR2_RT', 'NOT1STGEN_ENRL_ORIG_YR2_RT', 'NOT1STGEN_ENRL_4YR_TRANS_YR2_RT', 'NOT1STGEN_ENRL_2YR_TRANS_YR2_RT', 'NOT1STGEN_UNKN_ORIG_YR2_RT', 'NOT1STGEN_UNKN_4YR_TRANS_YR2_RT', 'NOT1STGEN_UNKN_2YR_TRANS_YR2_RT', 'DEATH_YR3_RT', 'COMP_ORIG_YR3_RT', 'COMP_4YR_TRANS_YR3_RT', 'COMP_2YR_TRANS_YR3_RT', 'WDRAW_ORIG_YR3_RT', 'WDRAW_4YR_TRANS_YR3_RT', 'WDRAW_2YR_TRANS_YR3_RT', 'ENRL_ORIG_YR3_RT', 'ENRL_4YR_TRANS_YR3_RT', 'ENRL_2YR_TRANS_YR3_RT', 'UNKN_ORIG_YR3_RT', 'UNKN_4YR_TRANS_YR3_RT', 'UNKN_2YR_TRANS_YR3_RT', 'LO_INC_DEATH_YR3_RT', 'LO_INC_COMP_ORIG_YR3_RT', 'LO_INC_COMP_4YR_TRANS_YR3_RT', 'LO_INC_COMP_2YR_TRANS_YR3_RT', 'LO_INC_WDRAW_ORIG_YR3_RT', 'LO_INC_WDRAW_4YR_TRANS_YR3_RT', 'LO_INC_WDRAW_2YR_TRANS_YR3_RT', 'LO_INC_ENRL_ORIG_YR3_RT', 'LO_INC_ENRL_4YR_TRANS_YR3_RT', 'LO_INC_ENRL_2YR_TRANS_YR3_RT', 'LO_INC_UNKN_ORIG_YR3_RT', 'LO_INC_UNKN_4YR_TRANS_YR3_RT', 'LO_INC_UNKN_2YR_TRANS_YR3_RT', 'MD_INC_DEATH_YR3_RT', 'MD_INC_COMP_ORIG_YR3_RT', 'MD_INC_COMP_4YR_TRANS_YR3_RT', 'MD_INC_COMP_2YR_TRANS_YR3_RT', 'MD_INC_WDRAW_ORIG_YR3_RT', 'MD_INC_WDRAW_4YR_TRANS_YR3_RT', 'MD_INC_WDRAW_2YR_TRANS_YR3_RT', 'MD_INC_ENRL_ORIG_YR3_RT', 'MD_INC_ENRL_4YR_TRANS_YR3_RT', 'MD_INC_ENRL_2YR_TRANS_YR3_RT', 'MD_INC_UNKN_ORIG_YR3_RT', 'MD_INC_UNKN_4YR_TRANS_YR3_RT', 'MD_INC_UNKN_2YR_TRANS_YR3_RT', 'HI_INC_DEATH_YR3_RT', 'HI_INC_COMP_ORIG_YR3_RT', 'HI_INC_COMP_4YR_TRANS_YR3_RT', 'HI_INC_COMP_2YR_TRANS_YR3_RT', 'HI_INC_WDRAW_ORIG_YR3_RT', 'HI_INC_WDRAW_4YR_TRANS_YR3_RT', 'HI_INC_WDRAW_2YR_TRANS_YR3_RT', 'HI_INC_ENRL_ORIG_YR3_RT', 'HI_INC_ENRL_4YR_TRANS_YR3_RT', 'HI_INC_ENRL_2YR_TRANS_YR3_RT', 'HI_INC_UNKN_ORIG_YR3_RT', 'HI_INC_UNKN_4YR_TRANS_YR3_RT', 'HI_INC_UNKN_2YR_TRANS_YR3_RT', 'DEP_DEATH_YR3_RT', 'DEP_COMP_ORIG_YR3_RT', 'DEP_COMP_4YR_TRANS_YR3_RT', 'DEP_COMP_2YR_TRANS_YR3_RT', 'DEP_WDRAW_ORIG_YR3_RT', 'DEP_WDRAW_4YR_TRANS_YR3_RT', 'DEP_WDRAW_2YR_TRANS_YR3_RT', 'DEP_ENRL_ORIG_YR3_RT', 'DEP_ENRL_4YR_TRANS_YR3_RT', 'DEP_ENRL_2YR_TRANS_YR3_RT', 'DEP_UNKN_ORIG_YR3_RT', 'DEP_UNKN_4YR_TRANS_YR3_RT', 'DEP_UNKN_2YR_TRANS_YR3_RT', 'IND_DEATH_YR3_RT', 'IND_COMP_ORIG_YR3_RT', 'IND_COMP_4YR_TRANS_YR3_RT', 'IND_COMP_2YR_TRANS_YR3_RT', 'IND_WDRAW_ORIG_YR3_RT', 'IND_WDRAW_4YR_TRANS_YR3_RT', 'IND_WDRAW_2YR_TRANS_YR3_RT', 'IND_ENRL_ORIG_YR3_RT', 'IND_ENRL_4YR_TRANS_YR3_RT', 'IND_ENRL_2YR_TRANS_YR3_RT', 'IND_UNKN_ORIG_YR3_RT', 'IND_UNKN_4YR_TRANS_YR3_RT', 'IND_UNKN_2YR_TRANS_YR3_RT', 'FEMALE_DEATH_YR3_RT', 'FEMALE_COMP_ORIG_YR3_RT', 'FEMALE_COMP_4YR_TRANS_YR3_RT', 'FEMALE_COMP_2YR_TRANS_YR3_RT', 'FEMALE_WDRAW_ORIG_YR3_RT', 'FEMALE_WDRAW_4YR_TRANS_YR3_RT', 'FEMALE_WDRAW_2YR_TRANS_YR3_RT', 'FEMALE_ENRL_ORIG_YR3_RT', 'FEMALE_ENRL_4YR_TRANS_YR3_RT', 'FEMALE_ENRL_2YR_TRANS_YR3_RT', 'FEMALE_UNKN_ORIG_YR3_RT', 'FEMALE_UNKN_4YR_TRANS_YR3_RT', 'FEMALE_UNKN_2YR_TRANS_YR3_RT', 'MALE_DEATH_YR3_RT', 'MALE_COMP_ORIG_YR3_RT', 'MALE_COMP_4YR_TRANS_YR3_RT', 'MALE_COMP_2YR_TRANS_YR3_RT', 'MALE_WDRAW_ORIG_YR3_RT', 'MALE_WDRAW_4YR_TRANS_YR3_RT', 'MALE_WDRAW_2YR_TRANS_YR3_RT', 'MALE_ENRL_ORIG_YR3_RT', 'MALE_ENRL_4YR_TRANS_YR3_RT', 'MALE_ENRL_2YR_TRANS_YR3_RT', 'MALE_UNKN_ORIG_YR3_RT', 'MALE_UNKN_4YR_TRANS_YR3_RT', 'MALE_UNKN_2YR_TRANS_YR3_RT', 'PELL_DEATH_YR3_RT', 'PELL_COMP_ORIG_YR3_RT', 'PELL_COMP_4YR_TRANS_YR3_RT', 'PELL_COMP_2YR_TRANS_YR3_RT', 'PELL_WDRAW_ORIG_YR3_RT', 'PELL_WDRAW_4YR_TRANS_YR3_RT', 'PELL_WDRAW_2YR_TRANS_YR3_RT', 'PELL_ENRL_ORIG_YR3_RT', 'PELL_ENRL_4YR_TRANS_YR3_RT', 'PELL_ENRL_2YR_TRANS_YR3_RT', 'PELL_UNKN_ORIG_YR3_RT', 'PELL_UNKN_4YR_TRANS_YR3_RT', 'PELL_UNKN_2YR_TRANS_YR3_RT', 'NOPELL_DEATH_YR3_RT', 'NOPELL_COMP_ORIG_YR3_RT', 'NOPELL_COMP_4YR_TRANS_YR3_RT', 'NOPELL_COMP_2YR_TRANS_YR3_RT', 'NOPELL_WDRAW_ORIG_YR3_RT', 'NOPELL_WDRAW_4YR_TRANS_YR3_RT', 'NOPELL_WDRAW_2YR_TRANS_YR3_RT', 'NOPELL_ENRL_ORIG_YR3_RT', 'NOPELL_ENRL_4YR_TRANS_YR3_RT', 'NOPELL_ENRL_2YR_TRANS_YR3_RT', 'NOPELL_UNKN_ORIG_YR3_RT', 'NOPELL_UNKN_4YR_TRANS_YR3_RT', 'NOPELL_UNKN_2YR_TRANS_YR3_RT', 'LOAN_DEATH_YR3_RT', 'LOAN_COMP_ORIG_YR3_RT', 'LOAN_COMP_4YR_TRANS_YR3_RT', 'LOAN_COMP_2YR_TRANS_YR3_RT', 'LOAN_WDRAW_ORIG_YR3_RT', 'LOAN_WDRAW_4YR_TRANS_YR3_RT', 'LOAN_WDRAW_2YR_TRANS_YR3_RT', 'LOAN_ENRL_ORIG_YR3_RT', 'LOAN_ENRL_4YR_TRANS_YR3_RT', 'LOAN_ENRL_2YR_TRANS_YR3_RT', 'LOAN_UNKN_ORIG_YR3_RT', 'LOAN_UNKN_4YR_TRANS_YR3_RT', 'LOAN_UNKN_2YR_TRANS_YR3_RT', 'NOLOAN_DEATH_YR3_RT', 'NOLOAN_COMP_ORIG_YR3_RT', 'NOLOAN_COMP_4YR_TRANS_YR3_RT', 'NOLOAN_COMP_2YR_TRANS_YR3_RT', 'NOLOAN_WDRAW_ORIG_YR3_RT', 'NOLOAN_WDRAW_4YR_TRANS_YR3_RT', 'NOLOAN_WDRAW_2YR_TRANS_YR3_RT', 'NOLOAN_ENRL_ORIG_YR3_RT', 'NOLOAN_ENRL_4YR_TRANS_YR3_RT', 'NOLOAN_ENRL_2YR_TRANS_YR3_RT', 'NOLOAN_UNKN_ORIG_YR3_RT', 'NOLOAN_UNKN_4YR_TRANS_YR3_RT', 'NOLOAN_UNKN_2YR_TRANS_YR3_RT', 'FIRSTGEN_DEATH_YR3_RT', 'FIRSTGEN_COMP_ORIG_YR3_RT', 'FIRSTGEN_COMP_4YR_TRANS_YR3_RT', 'FIRSTGEN_COMP_2YR_TRANS_YR3_RT', 'FIRSTGEN_WDRAW_ORIG_YR3_RT', 'FIRSTGEN_WDRAW_4YR_TRANS_YR3_RT', 'FIRSTGEN_WDRAW_2YR_TRANS_YR3_RT', 'FIRSTGEN_ENRL_ORIG_YR3_RT', 'FIRSTGEN_ENRL_4YR_TRANS_YR3_RT', 'FIRSTGEN_ENRL_2YR_TRANS_YR3_RT', 'FIRSTGEN_UNKN_ORIG_YR3_RT', 'FIRSTGEN_UNKN_4YR_TRANS_YR3_RT', 'FIRSTGEN_UNKN_2YR_TRANS_YR3_RT', 'NOT1STGEN_DEATH_YR3_RT', 'NOT1STGEN_COMP_ORIG_YR3_RT', 'NOT1STGEN_COMP_4YR_TRANS_YR3_RT', 'NOT1STGEN_COMP_2YR_TRANS_YR3_RT', 'NOT1STGEN_WDRAW_ORIG_YR3_RT', 'NOT1STGEN_WDRAW_4YR_TRANS_YR3_RT', 'NOT1STGEN_WDRAW_2YR_TRANS_YR3_RT', 'NOT1STGEN_ENRL_ORIG_YR3_RT', 'NOT1STGEN_ENRL_4YR_TRANS_YR3_RT', 'NOT1STGEN_ENRL_2YR_TRANS_YR3_RT', 'NOT1STGEN_UNKN_ORIG_YR3_RT', 'NOT1STGEN_UNKN_4YR_TRANS_YR3_RT', 'NOT1STGEN_UNKN_2YR_TRANS_YR3_RT', 'DEATH_YR4_RT', 'COMP_4YR_TRANS_YR4_RT', 'COMP_2YR_TRANS_YR4_RT', 'WDRAW_ORIG_YR4_RT', 'WDRAW_4YR_TRANS_YR4_RT', 'WDRAW_2YR_TRANS_YR4_RT', 'ENRL_ORIG_YR4_RT', 'ENRL_4YR_TRANS_YR4_RT', 'ENRL_2YR_TRANS_YR4_RT', 'UNKN_ORIG_YR4_RT', 'UNKN_4YR_TRANS_YR4_RT', 'UNKN_2YR_TRANS_YR4_RT', 'LO_INC_DEATH_YR4_RT', 'LO_INC_COMP_ORIG_YR4_RT', 'LO_INC_COMP_4YR_TRANS_YR4_RT', 'LO_INC_COMP_2YR_TRANS_YR4_RT', 'LO_INC_WDRAW_ORIG_YR4_RT', 'LO_INC_WDRAW_4YR_TRANS_YR4_RT', 'LO_INC_WDRAW_2YR_TRANS_YR4_RT', 'LO_INC_ENRL_ORIG_YR4_RT', 'LO_INC_ENRL_4YR_TRANS_YR4_RT', 'LO_INC_ENRL_2YR_TRANS_YR4_RT', 'LO_INC_UNKN_ORIG_YR4_RT', 'LO_INC_UNKN_4YR_TRANS_YR4_RT', 'LO_INC_UNKN_2YR_TRANS_YR4_RT', 'MD_INC_DEATH_YR4_RT', 'MD_INC_COMP_ORIG_YR4_RT', 'MD_INC_COMP_4YR_TRANS_YR4_RT', 'MD_INC_COMP_2YR_TRANS_YR4_RT', 'MD_INC_WDRAW_ORIG_YR4_RT', 'MD_INC_WDRAW_4YR_TRANS_YR4_RT', 'MD_INC_WDRAW_2YR_TRANS_YR4_RT', 'MD_INC_ENRL_ORIG_YR4_RT', 'MD_INC_ENRL_4YR_TRANS_YR4_RT', 'MD_INC_ENRL_2YR_TRANS_YR4_RT', 'MD_INC_UNKN_ORIG_YR4_RT', 'MD_INC_UNKN_4YR_TRANS_YR4_RT', 'MD_INC_UNKN_2YR_TRANS_YR4_RT', 'HI_INC_DEATH_YR4_RT', 'HI_INC_COMP_ORIG_YR4_RT', 'HI_INC_COMP_4YR_TRANS_YR4_RT', 'HI_INC_COMP_2YR_TRANS_YR4_RT', 'HI_INC_WDRAW_ORIG_YR4_RT', 'HI_INC_WDRAW_4YR_TRANS_YR4_RT', 'HI_INC_WDRAW_2YR_TRANS_YR4_RT', 'HI_INC_ENRL_ORIG_YR4_RT', 'HI_INC_ENRL_4YR_TRANS_YR4_RT', 'HI_INC_ENRL_2YR_TRANS_YR4_RT', 'HI_INC_UNKN_ORIG_YR4_RT', 'HI_INC_UNKN_4YR_TRANS_YR4_RT', 'HI_INC_UNKN_2YR_TRANS_YR4_RT', 'DEP_DEATH_YR4_RT', 'DEP_COMP_ORIG_YR4_RT', 'DEP_COMP_4YR_TRANS_YR4_RT', 'DEP_COMP_2YR_TRANS_YR4_RT', 'DEP_WDRAW_ORIG_YR4_RT', 'DEP_WDRAW_4YR_TRANS_YR4_RT', 'DEP_WDRAW_2YR_TRANS_YR4_RT', 'DEP_ENRL_ORIG_YR4_RT', 'DEP_ENRL_4YR_TRANS_YR4_RT', 'DEP_ENRL_2YR_TRANS_YR4_RT', 'DEP_UNKN_ORIG_YR4_RT', 'DEP_UNKN_4YR_TRANS_YR4_RT', 'DEP_UNKN_2YR_TRANS_YR4_RT', 'IND_DEATH_YR4_RT', 'IND_COMP_ORIG_YR4_RT', 'IND_COMP_4YR_TRANS_YR4_RT', 'IND_COMP_2YR_TRANS_YR4_RT', 'IND_WDRAW_ORIG_YR4_RT', 'IND_WDRAW_4YR_TRANS_YR4_RT', 'OVERALL_YR2_N', 'LO_INC_YR2_N', 'MD_INC_YR2_N', 'HI_INC_YR2_N', 'DEP_YR2_N', 'IND_YR2_N', 'FEMALE_YR2_N', 'MALE_YR2_N', 'PELL_YR2_N', 'NOPELL_YR2_N', 'LOAN_YR2_N', 'NOLOAN_YR2_N', 'FIRSTGEN_YR2_N', 'NOT1STGEN_YR2_N', 'OVERALL_YR3_N', 'LO_INC_YR3_N', 'MD_INC_YR3_N', 'HI_INC_YR3_N', 'DEP_YR3_N', 'IND_YR3_N', 'FEMALE_YR3_N', 'MALE_YR3_N', 'PELL_YR3_N', 'NOPELL_YR3_N', 'LOAN_YR3_N', 'NOLOAN_YR3_N', 'FIRSTGEN_YR3_N', 'NOT1STGEN_YR3_N', 'OVERALL_YR4_N', 'LO_INC_YR4_N', 'MD_INC_YR4_N', 'HI_INC_YR4_N', 'DEP_YR4_N', 'IND_YR4_N', 'FEMALE_YR4_N', 'MALE_YR4_N', 'PELL_YR4_N', 'NOPELL_YR4_N', 'LOAN_YR4_N', 'NOLOAN_YR4_N', 'FIRSTGEN_YR4_N', 'NOT1STGEN_YR4_N', 'OVERALL_YR6_N', 'LO_INC_YR6_N', 'MD_INC_YR6_N', 'HI_INC_YR6_N', 'DEP_YR6_N', 'IND_YR6_N', 'FEMALE_YR6_N', 'MALE_YR6_N', 'PELL_YR6_N', 'NOPELL_YR6_N', 'LOAN_YR6_N', 'NOLOAN_YR6_N', 'FIRSTGEN_YR6_N', 'NOT1STGEN_YR6_N', 'OVERALL_YR8_N', 'LO_INC_YR8_N', 'MD_INC_YR8_N', 'HI_INC_YR8_N', 'DEP_YR8_N', 'IND_YR8_N', 'FEMALE_YR8_N', 'MALE_YR8_N', 'PELL_YR8_N', 'NOPELL_YR8_N', 'LOAN_YR8_N', 'NOLOAN_YR8_N', 'FIRSTGEN_YR8_N', 'NOT1STGEN_YR8_N', 'SEPAR_DT_MDN', 'SEPAR_DT_N']
target_completion = target + completion_features
completion_df = processed_df[target_completion].copy()

completion_df.isna().sum()

UNITID                                  0
COMP_ORIG_YR4_RT                        0
C150_4                               6138
C150_L4                             13111
D150_4                               6138
D150_L4                             13111
C150_4_WHITE                        13321
C150_4_BLACK                        13562
C150_4_HISP                         13697
C150_4_ASIAN                        14263
C150_4_AIAN                         14953
C150_4_NHPI                         17573
C150_4_2MOR                         16884
C150_4_NRA                           9962
C150_4_UNKN                          9892
C150_4_WHITENH                      11460
C150_4_BLACKNH                      11807
C150_4_API                          12466
C150_L4_WHITE                       16084
C150_L4_BLACK                       16182
C150_L4_HISP                        16278
C150_L4_ASIAN                       16538
C150_L4_AIAN                        16623
C150_L4_NHPI                      

In [257]:
imputer8 = KNNImputer(n_neighbors=2, weights="uniform")
completion_KNN = pd.DataFrame(imputer8.fit_transform(completion_df), columns = list(completion_df.columns))

In [259]:
completion_KNN.isna().sum()

UNITID                              0
COMP_ORIG_YR4_RT                    0
C150_4                              0
C150_L4                             0
D150_4                              0
D150_L4                             0
C150_4_WHITE                        0
C150_4_BLACK                        0
C150_4_HISP                         0
C150_4_ASIAN                        0
C150_4_AIAN                         0
C150_4_NHPI                         0
C150_4_2MOR                         0
C150_4_NRA                          0
C150_4_UNKN                         0
C150_4_WHITENH                      0
C150_4_BLACKNH                      0
C150_4_API                          0
C150_L4_WHITE                       0
C150_L4_BLACK                       0
C150_L4_HISP                        0
C150_L4_ASIAN                       0
C150_L4_AIAN                        0
C150_L4_NHPI                        0
C150_L4_2MOR                        0
C150_L4_NRA                         0
C150_L4_UNKN

# Completion Correlation

In [260]:
completion_KNN['COMP_ORIG_YR4_RT'] = completion_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [275]:
corr_matrix1 = completion_KNN.corr()
completion_top5 = corr_matrix1['COMP_ORIG_YR4_RT']
completion_top5

,COMP_ORIG_YR4_RT,COMP_ORIG_YR4_RT
UNITID,0.003471,0.003471
COMP_ORIG_YR4_RT,1.000000,1.000000
C150_4,0.019392,0.019392
C150_L4,0.028206,0.028206
D150_4,-0.015947,-0.015947
D150_L4,-0.021195,-0.021195
C150_4_WHITE,0.012624,0.012624
C150_4_BLACK,0.020911,0.020911
C150_4_HISP,0.037378,0.037378
C150_4_ASIAN,0.018324,0.018324


In [ ]:
completion_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
completion_top5_list

# Repayment Features

In [175]:
repayment_features = ['CDR2', 'CDR3', 'DEATH_YR2_RT', 'COMP_ORIG_YR2_RT', 'COMP_4YR_TRANS_YR2_RT', 'COMP_2YR_TRANS_YR2_RT', 'WDRAW_ORIG_YR2_RT', 'WDRAW_4YR_TRANS_YR2_RT', 'WDRAW_2YR_TRANS_YR2_RT', 'ENRL_ORIG_YR2_RT', 'ENRL_4YR_TRANS_YR2_RT', 'ENRL_2YR_TRANS_YR2_RT', 'UNKN_ORIG_YR2_RT', 'UNKN_4YR_TRANS_YR2_RT', 'UNKN_2YR_TRANS_YR2_RT', 'LO_INC_DEATH_YR2_RT', 'LO_INC_COMP_ORIG_YR2_RT', 'LO_INC_COMP_4YR_TRANS_YR2_RT', 'LO_INC_COMP_2YR_TRANS_YR2_RT', 'LO_INC_WDRAW_ORIG_YR2_RT', 'LO_INC_WDRAW_4YR_TRANS_YR2_RT', 'LO_INC_WDRAW_2YR_TRANS_YR2_RT', 'LO_INC_ENRL_ORIG_YR2_RT', 'LO_INC_ENRL_4YR_TRANS_YR2_RT', 'LO_INC_ENRL_2YR_TRANS_YR2_RT', 'LO_INC_UNKN_ORIG_YR2_RT', 'LO_INC_UNKN_4YR_TRANS_YR2_RT', 'LO_INC_UNKN_2YR_TRANS_YR2_RT', 'MD_INC_DEATH_YR2_RT', 'MD_INC_COMP_ORIG_YR2_RT', 'MD_INC_COMP_4YR_TRANS_YR2_RT', 'MD_INC_COMP_2YR_TRANS_YR2_RT', 'MD_INC_WDRAW_ORIG_YR2_RT', 'MD_INC_WDRAW_4YR_TRANS_YR2_RT', 'MD_INC_WDRAW_2YR_TRANS_YR2_RT', 'MD_INC_ENRL_ORIG_YR2_RT', 'MD_INC_ENRL_4YR_TRANS_YR2_RT', 'MD_INC_ENRL_2YR_TRANS_YR2_RT', 'MD_INC_UNKN_ORIG_YR2_RT', 'MD_INC_UNKN_4YR_TRANS_YR2_RT', 'MD_INC_UNKN_2YR_TRANS_YR2_RT', 'HI_INC_DEATH_YR2_RT', 'HI_INC_COMP_ORIG_YR2_RT', 'HI_INC_COMP_4YR_TRANS_YR2_RT', 'HI_INC_COMP_2YR_TRANS_YR2_RT', 'HI_INC_WDRAW_ORIG_YR2_RT', 'HI_INC_WDRAW_4YR_TRANS_YR2_RT', 'HI_INC_WDRAW_2YR_TRANS_YR2_RT', 'HI_INC_ENRL_ORIG_YR2_RT', 'HI_INC_ENRL_4YR_TRANS_YR2_RT', 'HI_INC_ENRL_2YR_TRANS_YR2_RT', 'HI_INC_UNKN_ORIG_YR2_RT', 'HI_INC_UNKN_4YR_TRANS_YR2_RT', 'HI_INC_UNKN_2YR_TRANS_YR2_RT', 'DEP_DEATH_YR2_RT', 'DEP_COMP_ORIG_YR2_RT', 'DEP_COMP_4YR_TRANS_YR2_RT', 'DEP_COMP_2YR_TRANS_YR2_RT', 'DEP_WDRAW_ORIG_YR2_RT', 'DEP_WDRAW_4YR_TRANS_YR2_RT', 'DEP_WDRAW_2YR_TRANS_YR2_RT', 'DEP_ENRL_ORIG_YR2_RT', 'DEP_ENRL_4YR_TRANS_YR2_RT', 'DEP_ENRL_2YR_TRANS_YR2_RT', 'DEP_UNKN_ORIG_YR2_RT', 'DEP_UNKN_4YR_TRANS_YR2_RT', 'DEP_UNKN_2YR_TRANS_YR2_RT', 'IND_DEATH_YR2_RT', 'IND_COMP_ORIG_YR2_RT', 'IND_COMP_4YR_TRANS_YR2_RT', 'IND_COMP_2YR_TRANS_YR2_RT', 'IND_WDRAW_ORIG_YR2_RT', 'IND_WDRAW_4YR_TRANS_YR2_RT', 'IND_WDRAW_2YR_TRANS_YR2_RT', 'IND_ENRL_ORIG_YR2_RT', 'IND_ENRL_4YR_TRANS_YR2_RT', 'IND_ENRL_2YR_TRANS_YR2_RT', 'IND_UNKN_ORIG_YR2_RT', 'IND_UNKN_4YR_TRANS_YR2_RT', 'IND_UNKN_2YR_TRANS_YR2_RT', 'FEMALE_DEATH_YR2_RT', 'FEMALE_COMP_ORIG_YR2_RT', 'FEMALE_COMP_4YR_TRANS_YR2_RT', 'FEMALE_COMP_2YR_TRANS_YR2_RT', 'FEMALE_WDRAW_ORIG_YR2_RT', 'FEMALE_WDRAW_4YR_TRANS_YR2_RT', 'FEMALE_WDRAW_2YR_TRANS_YR2_RT', 'FEMALE_ENRL_ORIG_YR2_RT', 'FEMALE_ENRL_4YR_TRANS_YR2_RT', 'FEMALE_ENRL_2YR_TRANS_YR2_RT', 'FEMALE_UNKN_ORIG_YR2_RT', 'FEMALE_UNKN_4YR_TRANS_YR2_RT', 'FEMALE_UNKN_2YR_TRANS_YR2_RT', 'MALE_DEATH_YR2_RT', 'MALE_COMP_ORIG_YR2_RT', 'MALE_COMP_4YR_TRANS_YR2_RT', 'MALE_COMP_2YR_TRANS_YR2_RT', 'MALE_WDRAW_ORIG_YR2_RT', 'MALE_WDRAW_4YR_TRANS_YR2_RT', 'MALE_WDRAW_2YR_TRANS_YR2_RT', 'MALE_ENRL_ORIG_YR2_RT', 'MALE_ENRL_4YR_TRANS_YR2_RT', 'MALE_ENRL_2YR_TRANS_YR2_RT', 'MALE_UNKN_ORIG_YR2_RT', 'MALE_UNKN_4YR_TRANS_YR2_RT', 'MALE_UNKN_2YR_TRANS_YR2_RT', 'PELL_DEATH_YR2_RT', 'PELL_COMP_ORIG_YR2_RT', 'PELL_COMP_4YR_TRANS_YR2_RT', 'PELL_COMP_2YR_TRANS_YR2_RT', 'PELL_WDRAW_ORIG_YR2_RT', 'PELL_WDRAW_4YR_TRANS_YR2_RT', 'PELL_WDRAW_2YR_TRANS_YR2_RT', 'PELL_ENRL_ORIG_YR2_RT', 'PELL_ENRL_4YR_TRANS_YR2_RT', 'PELL_ENRL_2YR_TRANS_YR2_RT', 'PELL_UNKN_ORIG_YR2_RT', 'PELL_UNKN_4YR_TRANS_YR2_RT', 'PELL_UNKN_2YR_TRANS_YR2_RT', 'NOPELL_DEATH_YR2_RT', 'NOPELL_COMP_ORIG_YR2_RT', 'NOPELL_COMP_4YR_TRANS_YR2_RT', 'NOPELL_COMP_2YR_TRANS_YR2_RT', 'NOPELL_WDRAW_ORIG_YR2_RT', 'NOPELL_WDRAW_4YR_TRANS_YR2_RT', 'NOPELL_WDRAW_2YR_TRANS_YR2_RT', 'NOPELL_ENRL_ORIG_YR2_RT', 'NOPELL_ENRL_4YR_TRANS_YR2_RT', 'NOPELL_ENRL_2YR_TRANS_YR2_RT', 'NOPELL_UNKN_ORIG_YR2_RT', 'NOPELL_UNKN_4YR_TRANS_YR2_RT', 'NOPELL_UNKN_2YR_TRANS_YR2_RT', 'LOAN_DEATH_YR2_RT', 'LOAN_COMP_ORIG_YR2_RT', 'LOAN_COMP_4YR_TRANS_YR2_RT', 'LOAN_COMP_2YR_TRANS_YR2_RT', 'LOAN_WDRAW_ORIG_YR2_RT', 'LOAN_WDRAW_4YR_TRANS_YR2_RT', 'LOAN_WDRAW_2YR_TRANS_YR2_RT', 'LOAN_ENRL_ORIG_YR2_RT', 'LOAN_ENRL_4YR_TRANS_YR2_RT', 'LOAN_ENRL_2YR_TRANS_YR2_RT', 'LOAN_UNKN_ORIG_YR2_RT', 'LOAN_UNKN_4YR_TRANS_YR2_RT', 'LOAN_UNKN_2YR_TRANS_YR2_RT', 'NOLOAN_DEATH_YR2_RT', 'NOLOAN_COMP_ORIG_YR2_RT', 'NOLOAN_COMP_4YR_TRANS_YR2_RT', 'NOLOAN_COMP_2YR_TRANS_YR2_RT', 'NOLOAN_WDRAW_ORIG_YR2_RT', 'NOLOAN_WDRAW_4YR_TRANS_YR2_RT', 'NOLOAN_WDRAW_2YR_TRANS_YR2_RT', 'NOLOAN_ENRL_ORIG_YR2_RT', 'NOLOAN_ENRL_4YR_TRANS_YR2_RT', 'NOLOAN_ENRL_2YR_TRANS_YR2_RT', 'NOLOAN_UNKN_ORIG_YR2_RT', 'NOLOAN_UNKN_4YR_TRANS_YR2_RT', 'NOLOAN_UNKN_2YR_TRANS_YR2_RT', 'FIRSTGEN_DEATH_YR2_RT', 'FIRSTGEN_COMP_ORIG_YR2_RT', 'FIRSTGEN_COMP_4YR_TRANS_YR2_RT', 'FIRSTGEN_COMP_2YR_TRANS_YR2_RT', 'FIRSTGEN_WDRAW_ORIG_YR2_RT', 'FIRSTGEN_WDRAW_4YR_TRANS_YR2_RT', 'FIRSTGEN_WDRAW_2YR_TRANS_YR2_RT', 'FIRSTGEN_ENRL_ORIG_YR2_RT', 'FIRSTGEN_ENRL_4YR_TRANS_YR2_RT', 'FIRSTGEN_ENRL_2YR_TRANS_YR2_RT', 'FIRSTGEN_UNKN_ORIG_YR2_RT', 'FIRSTGEN_UNKN_4YR_TRANS_YR2_RT', 'FIRSTGEN_UNKN_2YR_TRANS_YR2_RT', 'NOT1STGEN_DEATH_YR2_RT', 'NOT1STGEN_COMP_ORIG_YR2_RT', 'NOT1STGEN_COMP_4YR_TRANS_YR2_RT', 'NOT1STGEN_COMP_2YR_TRANS_YR2_RT', 'NOT1STGEN_WDRAW_ORIG_YR2_RT', 'NOT1STGEN_WDRAW_4YR_TRANS_YR2_RT', 'NOT1STGEN_WDRAW_2YR_TRANS_YR2_RT', 'NOT1STGEN_ENRL_ORIG_YR2_RT', 'NOT1STGEN_ENRL_4YR_TRANS_YR2_RT', 'NOT1STGEN_ENRL_2YR_TRANS_YR2_RT', 'NOT1STGEN_UNKN_ORIG_YR2_RT', 'NOT1STGEN_UNKN_4YR_TRANS_YR2_RT', 'NOT1STGEN_UNKN_2YR_TRANS_YR2_RT']

In [176]:
target_repayment = target + repayment_features
repayment_df = processed_df[target_repayment].copy()
repayment_df.isna().sum()

UNITID                                  0
COMP_ORIG_YR4_RT                        0
CDR2                                  246
CDR3                                11712
DEATH_YR2_RT                         8101
COMP_ORIG_YR2_RT                     1276
COMP_4YR_TRANS_YR2_RT               10858
COMP_2YR_TRANS_YR2_RT                8345
WDRAW_ORIG_YR2_RT                     727
WDRAW_4YR_TRANS_YR2_RT               3493
WDRAW_2YR_TRANS_YR2_RT               3411
ENRL_ORIG_YR2_RT                      851
ENRL_4YR_TRANS_YR2_RT                2150
ENRL_2YR_TRANS_YR2_RT                3666
UNKN_ORIG_YR2_RT                     3408
UNKN_4YR_TRANS_YR2_RT               11723
UNKN_2YR_TRANS_YR2_RT               10280
LO_INC_DEATH_YR2_RT                  7683
LO_INC_COMP_ORIG_YR2_RT              6138
LO_INC_COMP_4YR_TRANS_YR2_RT        11669
LO_INC_COMP_2YR_TRANS_YR2_RT        11400
LO_INC_WDRAW_ORIG_YR2_RT             4810
LO_INC_WDRAW_4YR_TRANS_YR2_RT        9682
LO_INC_WDRAW_2YR_TRANS_YR2_RT     

In [177]:
for i in repayment_features:
    repayment_df[i] = repayment_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

C:\Users\Jesus Baquiax\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [178]:
repayment_df.isna().sum()

UNITID                                 0
COMP_ORIG_YR4_RT                       0
CDR2                                 233
CDR3                                 355
DEATH_YR2_RT                        2336
COMP_ORIG_YR2_RT                     378
COMP_4YR_TRANS_YR2_RT               5347
COMP_2YR_TRANS_YR2_RT               3728
WDRAW_ORIG_YR2_RT                    284
WDRAW_4YR_TRANS_YR2_RT              1266
WDRAW_2YR_TRANS_YR2_RT              1125
ENRL_ORIG_YR2_RT                     425
ENRL_4YR_TRANS_YR2_RT               1032
ENRL_2YR_TRANS_YR2_RT               1346
UNKN_ORIG_YR2_RT                    1225
UNKN_4YR_TRANS_YR2_RT               5622
UNKN_2YR_TRANS_YR2_RT               4143
LO_INC_DEATH_YR2_RT                 2308
LO_INC_COMP_ORIG_YR2_RT             2076
LO_INC_COMP_4YR_TRANS_YR2_RT        5413
LO_INC_COMP_2YR_TRANS_YR2_RT        5744
LO_INC_WDRAW_ORIG_YR2_RT            1842
LO_INC_WDRAW_4YR_TRANS_YR2_RT       4951
LO_INC_WDRAW_2YR_TRANS_YR2_RT       4182
LO_INC_ENRL_ORIG

In [182]:
imputer5 = KNNImputer(n_neighbors=2, weights="uniform")
Repayment_KNN = pd.DataFrame(imputer5.fit_transform(repayment_df), columns = list(repayment_df.columns))

In [183]:
Repayment_KNN.isna().sum()

UNITID                              0
COMP_ORIG_YR4_RT                    0
CDR2                                0
CDR3                                0
DEATH_YR2_RT                        0
COMP_ORIG_YR2_RT                    0
COMP_4YR_TRANS_YR2_RT               0
COMP_2YR_TRANS_YR2_RT               0
WDRAW_ORIG_YR2_RT                   0
WDRAW_4YR_TRANS_YR2_RT              0
WDRAW_2YR_TRANS_YR2_RT              0
ENRL_ORIG_YR2_RT                    0
ENRL_4YR_TRANS_YR2_RT               0
ENRL_2YR_TRANS_YR2_RT               0
UNKN_ORIG_YR2_RT                    0
UNKN_4YR_TRANS_YR2_RT               0
UNKN_2YR_TRANS_YR2_RT               0
LO_INC_DEATH_YR2_RT                 0
LO_INC_COMP_ORIG_YR2_RT             0
LO_INC_COMP_4YR_TRANS_YR2_RT        0
LO_INC_COMP_2YR_TRANS_YR2_RT        0
LO_INC_WDRAW_ORIG_YR2_RT            0
LO_INC_WDRAW_4YR_TRANS_YR2_RT       0
LO_INC_WDRAW_2YR_TRANS_YR2_RT       0
LO_INC_ENRL_ORIG_YR2_RT             0
LO_INC_ENRL_4YR_TRANS_YR2_RT        0
LO_INC_ENRL_

# Repayment Correlation

In [184]:
Repayment_KNN['COMP_ORIG_YR4_RT'] = Repayment_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [245]:
corr_matrix = Repayment_KNN.corr()
repayment_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
repayment_top5

COMP_ORIG_YR4_RT              1.000000
COMP_ORIG_YR2_RT              0.110795
LOAN_COMP_ORIG_YR2_RT         0.096380
MALE_COMP_ORIG_YR2_RT         0.091781
FEMALE_COMP_ORIG_YR2_RT       0.087515
NOT1STGEN_COMP_ORIG_YR2_RT    0.077709
Name: COMP_ORIG_YR4_RT, dtype: float64

In [248]:
repayment_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
repayment_top5_list

['COMP_ORIG_YR2_RT',
 'LOAN_COMP_ORIG_YR2_RT',
 'MALE_COMP_ORIG_YR2_RT',
 'FEMALE_COMP_ORIG_YR2_RT',
 'NOT1STGEN_COMP_ORIG_YR2_RT']

# Earnings Feature

In [189]:
earnings_features = ['count_nwne_p10', 'count_wne_p10', 'mn_earn_wne_p10', 'md_earn_wne_p10', 'pct10_earn_wne_p10', 'pct25_earn_wne_p10', 'pct75_earn_wne_p10', 'pct90_earn_wne_p10', 'sd_earn_wne_p10', 'count_wne_inc1_p10', 'count_wne_inc2_p10', 'count_wne_inc3_p10', 'count_wne_indep0_inc1_p10', 'count_wne_indep0_p10', 'count_wne_indep1_p10', 'count_wne_male0_p10', 'count_wne_male1_p10', 'gt_25k_p10', 'mn_earn_wne_inc1_p10', 'mn_earn_wne_inc2_p10', 'mn_earn_wne_inc3_p10', 'mn_earn_wne_indep0_inc1_p10', 'mn_earn_wne_indep0_p10', 'mn_earn_wne_indep1_p10', 'mn_earn_wne_male0_p10', 'mn_earn_wne_male1_p10', 'count_nwne_p6', 'count_wne_p6', 'mn_earn_wne_p6', 'md_earn_wne_p6', 'pct10_earn_wne_p6', 'pct25_earn_wne_p6', 'pct75_earn_wne_p6', 'pct90_earn_wne_p6', 'sd_earn_wne_p6', 'count_wne_inc1_p6', 'count_wne_inc2_p6', 'count_wne_inc3_p6', 'count_wne_indep0_inc1_p6', 'count_wne_indep0_p6', 'count_wne_indep1_p6', 'count_wne_male0_p6', 'count_wne_male1_p6', 'gt_25k_p6', 'mn_earn_wne_inc1_p6', 'mn_earn_wne_inc2_p6', 'mn_earn_wne_inc3_p6', 'mn_earn_wne_indep0_inc1_p6', 'mn_earn_wne_indep0_p6', 'mn_earn_wne_indep1_p6', 'mn_earn_wne_male0_p6', 'mn_earn_wne_male1_p6', 'count_nwne_p7', 'count_wne_p7', 'mn_earn_wne_p7', 'sd_earn_wne_p7', 'gt_25k_p7', 'count_nwne_p8', 'count_wne_p8', 'mn_earn_wne_p8', 'md_earn_wne_p8', 'pct10_earn_wne_p8', 'pct25_earn_wne_p8', 'pct75_earn_wne_p8', 'pct90_earn_wne_p8', 'sd_earn_wne_p8', 'gt_25k_p8', 'count_nwne_p9', 'count_wne_p9', 'mn_earn_wne_p9', 'sd_earn_wne_p9', 'gt_25k_p9']
target_earnings = target + earnings_features
earnings_df = processed_df[target_earnings].copy()
earnings_df.isna().sum()

UNITID                             0
COMP_ORIG_YR4_RT                   0
count_nwne_p10                  9450
count_wne_p10                   9451
mn_earn_wne_p10                 9615
md_earn_wne_p10                 9615
pct10_earn_wne_p10              9878
pct25_earn_wne_p10              9878
pct75_earn_wne_p10              9878
pct90_earn_wne_p10              9878
sd_earn_wne_p10                 9615
count_wne_inc1_p10              9568
count_wne_inc2_p10              9605
count_wne_inc3_p10             10141
count_wne_indep0_inc1_p10       9955
count_wne_indep0_p10            9634
count_wne_indep1_p10            9762
count_wne_male0_p10             9629
count_wne_male1_p10             9743
gt_25k_p10                      9615
mn_earn_wne_inc1_p10           11646
mn_earn_wne_inc2_p10           11643
mn_earn_wne_inc3_p10           11981
mn_earn_wne_indep0_inc1_p10    13281
mn_earn_wne_indep0_p10         11056
mn_earn_wne_indep1_p10         11019
mn_earn_wne_male0_p10          10495
m

In [190]:
for i in earnings_features:
    earnings_df[i] = earnings_df.groupby(['UNITID'], sort=False)[i].apply(lambda x: x.fillna(x.median()))

C:\Users\Jesus Baquiax\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [191]:
earnings_df.isna().sum()

UNITID                            0
COMP_ORIG_YR4_RT                  0
count_nwne_p10                  520
count_wne_p10                   526
mn_earn_wne_p10                 775
md_earn_wne_p10                 775
pct10_earn_wne_p10             1201
pct25_earn_wne_p10             1201
pct75_earn_wne_p10             1201
pct90_earn_wne_p10             1201
sd_earn_wne_p10                 775
count_wne_inc1_p10              707
count_wne_inc2_p10              747
count_wne_inc3_p10             1545
count_wne_indep0_inc1_p10      1247
count_wne_indep0_p10            806
count_wne_indep1_p10            924
count_wne_male0_p10             847
count_wne_male1_p10             947
gt_25k_p10                      775
mn_earn_wne_inc1_p10           3996
mn_earn_wne_inc2_p10           4025
mn_earn_wne_inc3_p10           4762
mn_earn_wne_indep0_inc1_p10    6767
mn_earn_wne_indep0_p10         3186
mn_earn_wne_indep1_p10         3158
mn_earn_wne_male0_p10          2286
mn_earn_wne_male1_p10       

In [192]:
imputer6 = KNNImputer(n_neighbors=2, weights="uniform")
earnings_KNN = pd.DataFrame(imputer6.fit_transform(earnings_df), columns = list(earnings_df.columns))

# Earnings Correlation

In [193]:
earnings_KNN['COMP_ORIG_YR4_RT'] = earnings_KNN['COMP_ORIG_YR4_RT'].astype(int)

In [194]:
corr_matrix = earnings_KNN.corr()
Repayment_top5 = corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6)
Repayment_top5

COMP_ORIG_YR4_RT         1.000000
pct10_earn_wne_p10       0.099295
mn_earn_wne_indep0_p6    0.092026
pct10_earn_wne_p8        0.088269
pct25_earn_wne_p6        0.087437
md_earn_wne_p8           0.086629
Name: COMP_ORIG_YR4_RT, dtype: float64

In [198]:
Repayment_top5_list = list(corr_matrix['COMP_ORIG_YR4_RT'].nlargest(n=6).index[1:])
Repayment_top5_list

['pct10_earn_wne_p10',
 'mn_earn_wne_indep0_p6',
 'pct10_earn_wne_p8',
 'pct25_earn_wne_p6',
 'md_earn_wne_p8']

# Feature / Correlation Summary

Below are the top five 

# Models

In [ ]:
X = processed_df[features]
y = processed_df['COMP_ORIG_YR4_RT']
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Baseline Model

In [ ]:
dr = DummyRegressor(strategy='median')
dr.fit(X_train, y_train)
dr.predict(X_train)
dr.score(X_train, y_train)

### First Model

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
linreg.score(X_train, y_train)